In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
            
            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])
            
            # img 28x28x1 (black/white)
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # L1 ImgIn shape=(?, 28, 28, 1)
            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
            
            L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L1 = tf.nn.dropout(L1, keep_prob=self.keep_prob)
            
            # L2 ImgIn shape=(?, 14, 14, 32)
            W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
            
            L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L2 = tf.nn.dropout(L2, keep_prob=self.keep_prob)
            
            # L3 ImgIn shape=(?, 7, 7, 64)
            W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
            
            L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L3 = tf.nn.dropout(L3, keep_prob=self.keep_prob)
            
            L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])
            
            # L4 Fully Connected Layer 4 * 4 * 128 inputs -> 625 outputs
            W4 = tf.get_variable("W4", shape = [128 * 4 * 4, 625], initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)
            
            # L5 Final FC 625 inputs -> 10 outputs
            W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.logits = tf.matmul(L4, W5) + b5
            
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            
            correct_prediction = tf.equal(tf.argmax(self.logits,1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
            
    def predict(self, x_test, keep_prob=1.0):
        return self.sess.run(self.logits, 
                             feed_dict={self.X: x_test, self.keep_prob: keep_prob})
    
    def get_accuracy(self, x_test, y_test, keep_prob=1.0):
        return self.sess.run(self.accuracy,
                            feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prob})
    
    def train(self, x_data, y_data, keep_prob=0.7):
        return self.sess.run([self.cost, self.optimizer], 
                             feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: keep_prob})

In [5]:
sess = tf.Session()

models = []
num_models = 10
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

## Ensemble training

In [8]:
sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)
    
print('Learning Finished!')

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print('Ensemble Accuracy:', sess.run(ensemble_accuracy))

Learning Started!
Epoch: 0001 cost = [0.44416016 0.42338538 0.51972621 0.37462941 0.41140284 0.37903619
 0.41166199 0.4196968  0.37931438 0.44122714]
Epoch: 0002 cost = [0.09529306 0.09591528 0.08922284 0.1003866  0.09929043 0.10070236
 0.0908242  0.09701668 0.09853249 0.09442774]
Epoch: 0003 cost = [0.07155774 0.07232798 0.06386882 0.07208786 0.0744542  0.0760129
 0.06493963 0.07115802 0.07244567 0.06942201]
Epoch: 0004 cost = [0.06021103 0.05776654 0.05180711 0.06079485 0.06204046 0.06417782
 0.05336903 0.05873786 0.06094326 0.0574154 ]
Epoch: 0005 cost = [0.05256251 0.05213467 0.04454627 0.05112329 0.05433941 0.05274705
 0.04972    0.05129889 0.0516603  0.04805816]
Epoch: 0006 cost = [0.04432859 0.04665918 0.04129861 0.04501207 0.04728005 0.04757883
 0.04354326 0.04862034 0.0461798  0.04468001]
Epoch: 0007 cost = [0.04272311 0.04225946 0.03674225 0.0441454  0.04173977 0.04219351
 0.03941707 0.04159767 0.04218021 0.03924361]
Epoch: 0008 cost = [0.03846021 0.03822807 0.03389096 0.0394